In [1]:
#

In [2]:
import json
import time
import numpy
import pandas
from scipy import stats
from data.flat import load
from matplotlib import pyplot
from matplotlib import lines as mlines

E:\venv\financial_news_re\lib\site-packages\pandas_datareader\compat\__init__.py:7: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  from pandas.util.testing import assert_frame_equal


In [3]:
from sklearn.metrics import r2_score
from sklearn.preprocessing import StandardScaler, MinMaxScaler, MaxAbsScaler, Normalizer
from sklearn.linear_model import LinearRegression

In [4]:
from m_utils.measures import r2_adj
from m_utils.sampling import ts_sampler
from m_utils.summarization import get_ols_summary
from m_utils.feature_selection import recursive_batch

In [5]:
d = './dataset.csv'
pandas.read_csv(d, nrows=100)

,ticker,time,open_LAG0,close_LAG0,high_LAG0,low_LAG0,volume_LAG0,open_LAG1,close_LAG1,high_LAG1,...,volume_LAG99,open_LAG100,close_LAG100,high_LAG100,low_LAG100,volume_LAG100,id,title,lag,news_time
0,MSFT,2019-09-23 15:53:00+00:00,-0.000359,0.000072,-0.000287,-0.000144,-0.370970,-0.000143,-0.000430,-0.000143,...,0.156438,0.000144,0.000361,0.000505,0.000217,-0.140647,244575,"""Flash Boys"" exchange IEX drops out of corpora...",1,2019-09-23 15:52:00+00:00
1,MSFT,2019-09-23 15:54:00+00:00,0.000144,0.000359,0.000502,0.000359,-0.138693,-0.000359,0.000072,-0.000287,...,-0.194598,0.000361,0.000577,0.000361,0.000361,0.156438,244575,"""Flash Boys"" exchange IEX drops out of corpora...",2,2019-09-23 15:52:00+00:00
2,MSFT,2019-09-23 15:55:00+00:00,0.000287,0.000215,0.000072,0.000215,-0.160793,0.000144,0.000359,0.000502,...,0.319666,0.000577,0.000433,0.000649,0.000505,-0.194598,244575,"""Flash Boys"" exchange IEX drops out of corpora...",3,2019-09-23 15:52:00+00:00
3,MSFT,2019-09-23 15:56:00+00:00,0.000287,0.000000,0.000215,0.000143,0.691880,0.000287,0.000215,0.000072,...,-0.100533,0.000577,0.000721,0.000648,0.000649,0.319666,244575,"""Flash Boys"" exchange IEX drops out of corpora...",4,2019-09-23 15:52:00+00:00
4,MSFT,2019-09-23 15:57:00+00:00,-0.000072,-0.000143,-0.000143,0.000000,-0.357331,0.000287,0.000000,0.000215,...,-0.415248,0.000576,0.000504,0.000648,0.000576,-0.100533,244575,"""Flash Boys"" exchange IEX drops out of corpora...",5,2019-09-23 15:52:00+00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,MSFT,2019-09-23 17:28:00+00:00,0.000431,0.000143,0.000143,0.000359,1.336280,-0.000215,0.000359,0.000287,...,-0.735656,0.000000,-0.000860,-0.000143,-0.000717,1.990911,244575,"""Flash Boys"" exchange IEX drops out of corpora...",96,2019-09-23 15:52:00+00:00
96,MSFT,2019-09-23 17:29:00+00:00,0.000143,0.000000,0.000072,0.000215,-0.132760,0.000431,0.000143,0.000143,...,0.683647,-0.000860,0.000287,-0.000502,-0.000072,-0.735656,244575,"""Flash Boys"" exchange IEX drops out of corpora...",97,2019-09-23 15:52:00+00:00
97,MSFT,2019-09-23 17:30:00+00:00,0.000000,0.000000,0.000000,-0.000215,-0.119770,0.000143,0.000000,0.000072,...,-0.489051,0.000502,-0.000287,0.000143,-0.000072,0.683647,244575,"""Flash Boys"" exchange IEX drops out of corpora...",98,2019-09-23 15:52:00+00:00
98,MSFT,2019-09-23 17:31:00+00:00,0.000072,0.000072,0.000143,0.000215,1.493580,0.000000,0.000000,0.000000,...,0.822350,-0.000573,-0.000143,-0.000573,-0.000359,-0.489051,244575,"""Flash Boys"" exchange IEX drops out of corpora...",99,2019-09-23 15:52:00+00:00


In [6]:
def load_data(data, tsi_names, y_names, removes):

    # clarify names

    exclude = y_names + removes
    x_names = [x for x in data.columns.values if (x not in exclude and 'LAG0' not in x)]
    
    # sample (without folds)

    kind = 'nofolds'
    test_rate = 0.2
    T_, X_, Y_ = numpy.array(tsi_names), numpy.array(x_names), numpy.array(y_names)
    T, X, Y = data[T_].values, data[X_].values, data[Y_].values
    T_train, X_train, Y_train, T_test, X_test, Y_test = ts_sampler(T, X, Y, None, test_rate, kind)
    
    return X_train, Y_train, X_test, Y_test, X_
    

In [7]:
def censory(model, XX, YY):
    if XX.shape[1] == 0:
        mask = numpy.array([False])
    else:
        p_values = get_ols_summary(model, XX, YY, list(range(XX.shape[1])))['Probabilities'].values[1:]
        mask = p_values < 0.05
    return mask

In [8]:
def golags(tsi_names, y_names, removes):
    report = pandas.DataFrame(columns=['R2_adj', 'd1'])
    X_train, Y_train, X_test, Y_test, X_ = load_data(data, tsi_names, y_names, removes)
    params = {'n_jobs': -1, 'fit_intercept': False}
    model = LinearRegression
    model_adj, X_train_adj, Y_train_adj, X_test_adj, Y_test_adj, X_adj_ = recursive_batch(model, params, X_train, Y_train, censory, X_test, Y_test, X_)
    Y_hat_train_adj = model_adj.predict(X_train_adj)
    Y_hat_test_adj = model_adj.predict(X_test_adj)
    result = {'R2_adj': r2_adj(Y_train_adj, Y_hat_train_adj, X_train_adj.shape[0], X_train_adj.shape[1]), 
              'd1': X_train_adj.shape[1]}
    report = report.append(result, ignore_index=True)
    print(report)
    return X_adj_

In [9]:
d = './dataset.csv'
data = pandas.read_csv(d)
data = data.rename(columns={'lag': 'news_horizon'})

tsi_names = ['news_time']
y_names = ['open_LAG0']
removes = ['ticker', 'time', 'id', 'title', 'news_time']

X_ = golags(tsi_names, y_names, removes)

    R2_adj     d1
0  0.92563  451.0


In [14]:
exclude = y_names + removes
x_names = [x for x in data.columns.values if (x not in exclude and 'LAG0' not in x)]
len(x_names)

501

In [10]:
X_.tolist()

['open_LAG1',
 'close_LAG1',
 'high_LAG1',
 'low_LAG1',
 'volume_LAG1',
 'open_LAG2',
 'close_LAG2',
 'high_LAG2',
 'low_LAG2',
 'volume_LAG2',
 'open_LAG3',
 'close_LAG3',
 'high_LAG3',
 'low_LAG3',
 'volume_LAG3',
 'open_LAG4',
 'close_LAG4',
 'high_LAG4',
 'low_LAG4',
 'volume_LAG4',
 'open_LAG5',
 'close_LAG5',
 'high_LAG5',
 'low_LAG5',
 'volume_LAG5',
 'open_LAG6',
 'close_LAG6',
 'high_LAG6',
 'low_LAG6',
 'volume_LAG6',
 'open_LAG7',
 'close_LAG7',
 'high_LAG7',
 'low_LAG7',
 'volume_LAG7',
 'open_LAG8',
 'close_LAG8',
 'high_LAG8',
 'low_LAG8',
 'volume_LAG8',
 'open_LAG9',
 'close_LAG9',
 'high_LAG9',
 'low_LAG9',
 'open_LAG10',
 'close_LAG10',
 'high_LAG10',
 'low_LAG10',
 'volume_LAG10',
 'open_LAG11',
 'close_LAG11',
 'high_LAG11',
 'low_LAG11',
 'open_LAG12',
 'close_LAG12',
 'high_LAG12',
 'low_LAG12',
 'volume_LAG12',
 'open_LAG13',
 'close_LAG13',
 'high_LAG13',
 'low_LAG13',
 'volume_LAG13',
 'open_LAG14',
 'close_LAG14',
 'high_LAG14',
 'low_LAG14',
 'volume_LAG14',
